## CUTEst* and NLPModels.jl

The CUTEst project is the modern and modular thread-safe predecessor of the CUTEr project, which was the most adopted Fortran package in the evolution of _Constrained and Unconstrained Testing Environment (CUTE)_ family for optimization problems. In addition to thread safety, CUTEst features dynamic memory allocation.
The CUTEst additions were facilitated by a massive overhaul of the CUTEr's Fortran 77 codebase to Fortran 2003 release.
Most of the changes where internal, and there is very little documentation for CUTEst. Instead, CUTEst generally refers you to the old CUTEr documentation.
 + Source - 9ish pages discussing the CUTEst updates: https://www.researchgate.net/publication/265164342_CUTEst_a_Constrained_and_Unconstrained_Testing_Environment_with_safe_threads_for_Mathematical_Optimization
 
+ Source - the CUTEst GitHub repo, fairly wordy and useless. The docs directory is split up into a bunch of pdf's containing references to the Fortran CUTEr interface: https://github.com/ralna/CUTEst


As for CUTEst.jl, it is a wrapper of the CUTEst project that extends the NLPModels.jl abstract Julia type, which attempts to standardize the interface for representing optimization problems in Julia.  
The CUTEst.jl documentation is lacking; instead, it refers you to the CUTEst Fortran project (GitHub repo referenced above).
The NLPModels documentation is quite good, so I am assuming CUTEst.jl implements everything needed to fulfill its subtype requirements.
(See Basic requirements of the NLPModels docs:https://juliasmoothoptimizers.github.io/NLPModels.jl/dev/)


***Question*** Where does AD fit into all this? After reading the AD wiki and scanning the paper cited in the ForwardDiff.jl package, it makes so much sense!
What a great alternative and paradigm shift for computing differentials.
The NLPModels.jl package says that programming problems must implement a grad() and a hess() method that accepts and NLPModels type, which CUTEST.jl does (as shown below). 
However, it is unclear if the CUTEST.jl package is using an Algorithmic/Automatic differentiation in their implementations.
After scanning CUTEst.jl sourcecode, they make a lot of _ccall()'s_, which makes sense since it is a wrapper. I suppose they are calling a wrapper of the Fortran package, namely the C interface.
So the question becomes, does the original CUTEst use AD?



In [8]:
using CUTEst, NLPModels

# from the README.md of CUTEst julia package
nlp = CUTEstModel("ROSENBR")
println("x0 = $(nlp.meta.x0)")
println("fx = $( obj(nlp, nlp.meta.x0) )")
println("gx = $( grad(nlp, nlp.meta.x0) )")
println("Hx = $( hess(nlp, nlp.meta.x0) )")
problems = CUTEst.select()
display(nlp)
typeof(nlp)
finalize(nlp)

  Problem name: ROSENBR
   All variables: ████████████████████ 2      All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ████████████████████ 2                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (  0.00% sparsity)   3               linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                    nonlinear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                                                         nnzj: (------% sparsity)         



x0 = [-1.2, 1.0]
fx = 24.199999999999996
gx = [-215.59999999999997, -87.99999999999999]
Hx = [1330.0 480.0; 480.0 200.0]


## JuliaGPU

As for computing on the GPU, it isn't going to work on my Mac. I have a Nvidia GeForce graphics card, which once supported CUDA. However, Apple and Nvidia got into it a while back, and Apple now has a bunch of system restrictions that no longer give you access to the Nvidia GPU on their most recent updates. (it is pretty comical from my reading) I tried to boot my machine in recover mode, override system privileges, and run an open-source CUDA driver executable, but it did not grant access to the GPU.

It shouldn't be too much of an issue to tell Julia you have GPU on a Linux or Windows operating system (Assuming it is a nice ARM, Nvidia or Intel GPU). However, JuliaHub appears to be a great platform that allows for Cloud deployment and has all the backend taken care of, so it is as easy as putting _Pkg.add(JuliaGPU), using JuliaGPU_ in your notebook. It costs about $0.32/per hour at a minimum and scales up rapidly when you try to distribute the load. 